### Imports and variables

In [ ]:
import pandas as pd

from lmkgroup_ds_utils.db.connector import DB
from cheffelo_personalization.basket_preselector.utils.paths import SQL_DIR
from cheffelo_personalization.basket_preselector.data.models.customer import Customer

read_db = DB(db_name = 'analytics_db', local=True)

In [ ]:
# Product types
MEALBOX_PRODUCT_TYPE_ID = "2f163d69-8ac1-6e0c-8793-ff0000804eb3"
FLEX_PRODUCT_TYPE_ID = "cac333ea-ec15-4eea-9d8d-2b9ef60ec0c1"

# Preference types
CONCEPT_PREFERENCES = "009cf63e-6e84-446c-9ce4-afdbb6bb9687"
TASTE_PREFERENCE = "4c679266-7dc0-4a8e-b72d-e9bb8dadc7eb"

In [ ]:
agreement_id = 1235109

In [ ]:
company_id = "6a2d0b60-84d6-4830-9945-58d518d27ac2"
weeks = {
    "week": 31,
    "year": 2023
}

customer_status = 10

### Get menu with product information

#### Product information with attributes

In [ ]:
get_product_information = """
    -- to avoid parameter sniffing
    declare @company uniqueidentifier
    select @company = '{company_id}'

    SELECT
        p.id AS product_id
    ,	p.name AS product_name
    ,	pt.product_type_name AS product_type
    ,	pv.id AS variation_id
    ,	pv.sku AS variation_sku
    ,	pvc.name AS variation_name
    ,	pvc.company_id AS company_id
    ,	ISNULL(pvav.attribute_value, pvat.default_value) AS variation_meals
    ,	ISNULL(pvav2.attribute_value, pvat2.default_value) AS variation_portions
    ,	ISNULL(pvav4.attribute_value, pvat4.default_value) AS variation_price
    ,	ROUND(CAST(ISNULL(pvav4.attribute_value, pvat4.default_value) AS FLOAT) * (1.0 + (CAST(ISNULL(pvav5.attribute_value, pvat5.default_value) AS FLOAT) / 100)), 2) AS variation_price_incl_vat
    ,	ISNULL(pvav5.attribute_value, pvat5.default_value) AS variation_vat
    FROM product_layer.product p
    INNER JOIN product_layer.product_type pt ON pt.product_type_id = p.product_type_id
    INNER JOIN product_layer.product_variation pv ON pv.product_id = p.id
    INNER JOIN product_layer.product_variation_company pvc ON pvc.variation_id = pv.id
    INNER JOIN cms.company c ON c.id = pvc.company_id

    -- MEALS
    LEFT JOIN product_layer.product_variation_attribute_template pvat ON pvat.product_type_id = p.product_type_id AND pvat.attribute_name = 'Meals'
    LEFT JOIN product_layer.product_variation_attribute_value pvav ON pvav.attribute_id = pvat.attribute_id AND pvav.variation_id = pvc.variation_id AND pvav.company_id = pvc.company_id

    -- PORTIONS
    LEFT JOIN product_layer.product_variation_attribute_template pvat2 ON pvat2.product_type_id = p.product_type_id AND pvat2.attribute_name = 'Portions'
    LEFT JOIN product_layer.product_variation_attribute_value pvav2 ON pvav2.attribute_id = pvat2.attribute_id AND pvav2.variation_id = pvc.variation_id AND pvav2.company_id = pvc.company_id

    -- PRICE
    LEFT JOIN product_layer.product_variation_attribute_template pvat4 ON pvat4.product_type_id = p.product_type_id AND pvat4.attribute_name = 'PRICE'
    LEFT JOIN product_layer.product_variation_attribute_value pvav4 ON pvav4.attribute_id = pvat4.attribute_id AND pvav4.variation_id = pvc.variation_id AND pvav4.company_id = pvc.company_id

    -- VAT
    LEFT JOIN product_layer.product_variation_attribute_template pvat5 ON pvat5.product_type_id = p.product_type_id AND pvat5.attribute_name = 'VAT'
    LEFT JOIN product_layer.product_variation_attribute_value pvav5 ON pvav5.attribute_id = pvat5.attribute_id AND pvav5.variation_id = pvc.variation_id AND pvav5.company_id = pvc.company_id

    WHERE pvc.company_id=@company
""".format(company_id=company_id)

In [ ]:
df_products = read_db.read_data(get_product_information)
df_products.head()

#### Get concept and taste preferences for products

In [ ]:
get_recipe_preferences_query = """
declare @company uniqueidentifier = '{company_id}', @week int = '{week}', @year int = '{year}';

WITH distinct_recipe_preferences AS (
    SELECT DISTINCT rp.recipe_portion_id, rmt.recipe_name, p.preference_id, p.name
        FROM pim.weekly_menus wm
        INNER JOIN pim.menus m ON m.weekly_menus_id = wm.weekly_menus_id AND m.product_type_id in ('CAC333EA-EC15-4EEA-9D8D-2B9EF60EC0C1', '2F163D69-8AC1-6E0C-8793-FF0000804EB3')--Velg&Vrak dishes
        INNER JOIN pim.menu_variations mv ON mv.menu_id = m.menu_id
        INNER JOIN pim.menu_recipes mr ON mr.MENU_ID = m.menu_id AND mr.MENU_RECIPE_ORDER <= mv.MENU_NUMBER_DAYS
        INNER JOIN pim.recipes r ON r.recipe_id = mr.RECIPE_ID
        INNER JOIN cms.company c ON c.id = wm.company_id
        INNER JOIN cms.country cont ON cont.id = c.country_id
        INNER JOIN pim.recipe_metadata_translations rmt ON rmt.recipe_metadata_id = r.recipe_metadata_id AND rmt.language_id = cont.default_language_id
        INNER JOIN pim.recipe_portions rp ON rp.RECIPE_ID = r.recipe_id AND rp.portion_id = mv.PORTION_ID
        INNER JOIN pim.portions pt ON pt.PORTION_ID = rp.PORTION_ID
        INNER JOIN pim.chef_ingredient_sections cis ON cis.RECIPE_PORTION_ID = rp.recipe_portion_id
        INNER JOIN pim.chef_ingredients ci ON ci.CHEF_INGREDIENT_SECTION_ID = cis.CHEF_INGREDIENT_SECTION_ID
        INNER JOIN pim.order_ingredients oi ON oi.ORDER_INGREDIENT_ID = ci.ORDER_INGREDIENT_ID
        INNER JOIN pim.ingredients i ON i.ingredient_internal_reference = oi.INGREDIENT_INTERNAL_REFERENCE
        INNER JOIN pim.find_ingredient_categories_parents icc ON icc.ingredient_id = i.ingredient_id
        INNER JOIN pim.ingredient_category_preference icp ON icp.ingredient_category_id = icc.parent_category_id
        INNER JOIN cms.preference p ON p.preference_id = icp.preference_id
        AND p.preference_type_id = '4C679266-7DC0-4A8E-B72D-E9BB8DADC7EB'
        INNER JOIN cms.preference_company pc ON pc.company_id = wm.company_id AND pc.preference_id = p.preference_id AND pc.is_active = 1
        where m.RECIPE_STATE = 1 AND wm.menu_week = @week AND wm.menu_year = @year AND wm.company_id = @company
)
SELECT
    recipe_portion_id,
    STRING_AGG(convert(nvarchar(36), preference_id), ', ') as preference_ids,
    STRING_AGG(name, ', ') as preferences
    FROM distinct_recipe_preferences
    GROUP BY recipe_portion_id
""".format(company_id=company_id, week=weeks["week"], year=weeks["year"])

In [ ]:
df_recipe_preferences = read_db.read_data(get_recipe_preferences_query)
df_recipe_preferences.head()

#### Get menu for given year, week and company

In [ ]:
get_menu_for_yearweek_company = """
declare @company uniqueidentifier, @week int, @year int
select @company = '{company_id}', @week = '{week}', @year = '{year}'
    
SELECT
    wm.menu_year as year
    ,	wm.menu_week as week
    ,	c.company_name
    ,	m.menu_name as product
    ,   m.product_type_id
    ,	mv.variation_name as variation
    ,   mv.MENU_VARIATION_EXT_ID as variation_id
    ,   mr.RECIPE_ID as menu_recipe
    ,   r.main_recipe_id as recipe_id
    ,   rp.recipe_portion_id
    FROM pim.weekly_menus wm
        INNER JOIN pim.menus m ON m.weekly_menus_id = wm.weekly_menus_id
        AND m.product_type_id in ('CAC333EA-EC15-4EEA-9D8D-2B9EF60EC0C1', '2F163D69-8AC1-6E0C-8793-FF0000804EB3')
        INNER JOIN pim.menu_variations mv ON mv.menu_id = m.menu_id
        INNER JOIN pim.menu_recipes mr ON mr.MENU_ID = m.menu_id
        INNER JOIN pim.recipes r ON r.recipe_id = mr.RECIPE_ID
        INNER JOIN pim.recipe_portions rp ON rp.RECIPE_ID = r.recipe_id AND rp.portion_id = mv.PORTION_ID
        INNER JOIN cms.company c ON c.id = wm.company_id
        WHERE m.RECIPE_STATE = 1
            AND wm.menu_year = @year 
            AND wm.menu_week = @week
            AND c.id = @company
        order by wm.menu_year, wm.menu_week, mv.MENU_VARIATION_EXT_ID
""".format(company_id=company_id, week=weeks["week"], year=weeks["year"])

In [ ]:
df_menu = read_db.read_data(get_menu_for_yearweek_company)
df_menu.head()

#### Merge product information to menu

In [ ]:
df_menu_products = df_menu.merge(df_products, on="variation_id", how="left")
df_menu_products.head()

In [ ]:
df_menu_products.columns

In [ ]:
df_flex_dishes = df_menu_products[df_menu_products["product_type_id"] == FLEX_PRODUCT_TYPE_ID.upper()]
print(len(df_flex_dishes))
df_flex_dishes.head()

In [ ]:
df_menu_mealboxes = df_menu_products[df_menu_products["product_type_id"] == MEALBOX_PRODUCT_TYPE_ID.upper()]
print(len(df_mealboxes))
df_menu_mealboxes.head()

### Merge preference information to menu products

In [ ]:
df_menu_products_with_preferences = df_flex_dishes.merge(df_recipe_preferences, how="left", on="recipe_portion_id")
df_menu_products_with_preferences.head()

In [ ]:
len(df_menu_products_with_preferences)

In [ ]:
df_menu_products_with_preferences

### Get general customer information

In [ ]:
# TODO: Add subscription product type id and the product id to fetch the portion size

In [ ]:
customers_information_for_company_query = """
    declare @company uniqueidentifier = '{company_id}';
    declare @status varchar(20) = '{status}';
    declare @concept_preference_id uniqueidentifier = '{concept_preference_id}';
    declare @taste_preference_id uniqueidentifier = '{taste_preference_id}';

    WITH concept_preferences AS (
        SELECT
            bap.agreement_id,
            bap.preference_id,
            pref.name preference_name,
            bap.priority
        from cms.billing_agreement_preference bap
            JOIN cms.preference pref on pref.preference_id = bap.preference_id
            WHERE pref.preference_type_id = @concept_preference_id
    ), taste_preferences AS (
        SELECT
            bap.agreement_id,
            bap.preference_id,
            pref.name preference_name,
            bap.priority
        from cms.billing_agreement_preference bap
            JOIN cms.preference pref on pref.preference_id = bap.preference_id
            WHERE pref.preference_type_id = @taste_preference_id
    )

    SELECT ba.agreement_id
            , ba.company_id
            , ba.status
            , taste_pref.preference_id taste_preference_id
            , taste_pref.preference_name taste_preference_name
            , concept_pref.preference_id concept_preference_id
            , concept_pref.preference_name concept_preference_name
            , bap.subscribed_product_variation_id
            , bap.quantity
        FROM cms.billing_agreement ba
        LEFT JOIN taste_preferences taste_pref on taste_pref.agreement_id = ba.agreement_id
        LEFT JOIN concept_preferences concept_pref on concept_pref.agreement_id = ba.agreement_id
        LEFT JOIN cms.billing_agreement_basket bb on bb.agreement_id = ba.agreement_id
        LEFT JOIN cms.billing_agreement_basket_product bap on bap.billing_agreement_basket_id = bb.id
        WHERE status = @status and company_id = @company
""".format(status=customer_status, company_id=company_id, concept_preference_id=CONCEPT_PREFERENCES, taste_preference_id=TASTE_PREFERENCE)

In [ ]:
df_customer_information = read_db.read_data(customers_information_for_company_query).dropna(subset=["taste_preference_id", "concept_preference_id"], how='all')

In [ ]:
df_customer_information.head()

#### Find mealbox variation

In [ ]:
df_customer_information_filtered = (
    df_customer_information[df_customer_information["subscribed_product_variation_id"]
                            .isin(df_menu_mealboxes["variation_id"])]
)

In [ ]:
df_customers = df_customer_information_filtered.merge(df_menu_mealboxes, right_on="variation_id", left_on="subscribed_product_variation_id")

In [ ]:
df_customers.head()

#### For customers, get their planned baskets

In [ ]:
# Skipping this for now

#### For customers, get their already delivered orders

In [ ]:
# Skipping this for now